In [105]:
import re
import numpy as np
from sys import maxsize
import math

def validate_input(reference_sequence, new_sequence, allowed_characters):
    """ This function validates that the input sequences contain only allowed characters."""
    for character in reference_sequence:
        if character.upper() not in allowed_characters:
            raise Exception('incorrect character in reference sequence')

    for character in new_sequence:
        if character.upper() not in allowed_characters:
            raise Exception('incorrect character in new sequence')

            
def init_alignment_matrix(reference_sequence, new_sequence, gap_penalty):
    alignment_matrix = [[[0 for x in range(len(new_sequence))]for y in range(len(reference_sequence))]for z in range(2)]

    for x in range(len(new_sequence)):
                        alignment_matrix[0][0][x]=new_sequence[x]
                        alignment_matrix[1][0][x]=new_sequence[x]
    for y in range(len(reference_sequence)):
                        alignment_matrix[0][y][0]=reference_sequence[y]
                        alignment_matrix[1][y][0]=reference_sequence[y]

    for x in range(len(new_sequence)-2):
                        alignment_matrix[0][1][x+2]=(x+1)*-gap_penalty
    for y in range(len(reference_sequence)-2):
                        alignment_matrix[0][y+2][1]=(y+1)*-gap_penalty 
    return alignment_matrix


def get_list_of_max_indices(values_list):
    values_list2 = sorted(values_list, key=lambda value: value[0], reverse=True)
    result = []
    prev_value = None
    for value in values_list2:
        if prev_value is not None and value[0] < prev_value:
            break
        result.append(value)
        prev_value = value[0]
    return result


def calc_simple_score(reference_sequence, new_sequence, similarity_score_fn, gap_penalty):
    score = 0
    for a, b in zip(reference_sequence, new_sequence):
        if '-' in [a, b]:
            score -= gap_penalty
        else:
            score += similarity_score_fn(a, b)
    return score

    
def calc_score(i, j, matrix, reference_sequence, new_sequence, similarity_score_fn, gap_penalty):
    #print(f'{i},{j}')
    if matrix[i, j] != -maxsize:
        #print(f'b1')
        return matrix[i, j]
    elif i == j:
        #print(f'b2')
        a = reference_sequence[i]
        b = new_sequence[i]
        if '-' in [a, b]:
            return -gap_penalty
        else:
            return similarity_score_fn(a, b)
    else:
        #print(f'b3')
        pivot = int(math.floor((j-i)/2) + i)
        matrix[i, pivot] = calc_score(i, pivot, matrix, reference_sequence, new_sequence, similarity_score_fn, gap_penalty)
        matrix[pivot + 1, j] = calc_score(pivot+1, j, matrix, reference_sequence, new_sequence, similarity_score_fn, gap_penalty)
        return matrix[i, pivot] + matrix[pivot + 1, j]
        

def calc_seq_similarity_score(reference_sequence, new_sequence, similarity_score_fn, gap_penalty, max_len=None):
    assert(len(reference_sequence) == len(new_sequence))
    
    if max_len is None:
        max_len = len(reference_sequence)
        
    matrix = np.zeros((len(reference_sequence), len(new_sequence)), np.int)
    matrix.fill(-maxsize)
    
    max_score = None
    indices = (None, None)
    for i in range(len(reference_sequence)):
        j = i + max_len - 1
        if j >= len(reference_sequence):
            break
        #print(f'{i},{j}')
        score = calc_score(i, j, matrix, reference_sequence, new_sequence, similarity_score_fn, gap_penalty)
        if max_score is None or score > max_score:
            max_score = score
            indices = (i, j)

    return (max_score, indices)
        

def align_sequences(reference_sequence, new_sequence, allowed_characters, gap_penalty, similarity_score_fn):
    """ This function calculates the best alignments of two sequences. """

    # Normalise sequences
    reference_sequence = reference_sequence.upper()
    new_sequence = new_sequence.upper()
    
    validate_input(reference_sequence, new_sequence, allowed_characters)

    # Add zero prefix
    reference_sequence = '00' + reference_sequence
    new_sequence = '00' + new_sequence

    alignment_matrix = init_alignment_matrix(reference_sequence, new_sequence, gap_penalty)
    
    j = 0
    i = 0
    h = 0
    for x in range(len(new_sequence)-2):
        for y in range(len(reference_sequence)-2):
            sxy = similarity_score(reference_sequence[y+2], new_sequence[x+2])
            alignment_matrix[1][y+2][x+2] = sxy

            diag  =  sxy + alignment_matrix[0][y+2-1][x+2-1]
            right =  alignment_matrix[0][y+2][x+2-1] - gap_penalty
            down  =  alignment_matrix[0][y+2-1][x+2] - gap_penalty
            if  diag > right and diag > down :
                h = diag
            elif right > down:
                h = right
            else:
                h = down
            alignment_matrix[0][y+2][x+2]= h

    x = len(new_sequence) - 1
    y = len(reference_sequence) - 1
    previous_choice = 'none'
    aligned_reference_sequence = ''
    aligned_new_sequence = ''
    
    stack = [] #[(y, x, "ref_seq", "new_seq")]
    while True:
        #choices = ''
        
        if x == 2 and y == 2:
            if len(stack) > 0:
                print(f'> {aligned_reference_sequence}\n> {aligned_new_sequence}')
                seq_similarity_score_value, (best_indices) = calc_seq_similarity_score(aligned_reference_sequence,
                                                                                       aligned_new_sequence,
                                                                                       similarity_score, gap_penalty,
                                                                                       len(new_sequence))
                print(f'similarity_score={seq_similarity_score_value}, best_indices={best_indices}')
                simple_score = calc_simple_score(aligned_reference_sequence,
                                                 aligned_new_sequence,
                                                 similarity_score, gap_penalty)
                print(f'simple_score={simple_score}')
                print('\n')
                y, x, aligned_reference_sequence, aligned_new_sequence = stack.pop()
            else:
                break
        elif x == 2:
            aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
            aligned_new_sequence =  '-' + aligned_new_sequence
            x = x
            y = y-1

        elif y == 2:
            aligned_reference_sequence =  '-' + aligned_reference_sequence
            aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
            x = x-1
            y = y

        else:
            #print(f'y={y}, x={x}')
            list_of_indeces = get_list_of_max_indices([(alignment_matrix[0][y-1][x-1], y-1, x-1, 'd'),
                                                       (alignment_matrix[0][y-1][x], y-1, x, 'u'),
                                                       (alignment_matrix[0][y][x-1], y, x-1, 'l')])
            assert(len(list_of_indeces) > 0)
            
            prev_aligned_reference_sequence = aligned_reference_sequence
            prev_aligned_new_sequence = aligned_new_sequence

            choice = list_of_indeces[0][3]
            if choice == 'u':
                aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
                aligned_new_sequence =  '-' + aligned_new_sequence
                x = x
                y = y-1
            elif choice == 'd':
                aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
                aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
                x = x-1
                y = y-1
            elif choice == 'l':
                aligned_reference_sequence =  '-' + aligned_reference_sequence
                aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
                x = x-1
                y = y
            else:
                raise Exception('error at choice')
            #previous_choice = choice
            
            if len(list_of_indeces) > 1:
                for value in list_of_indeces[1:]:
                    choice = value[3]
                    if choice == 'u':
                        stack_aligned_reference_sequence =  reference_sequence[value[1]] + prev_aligned_reference_sequence
                        stack_aligned_new_sequence =  '-' + prev_aligned_new_sequence
                    elif choice == 'd':
                        stack_aligned_reference_sequence =  reference_sequence[value[1]] + prev_aligned_reference_sequence
                        stack_aligned_new_sequence =  new_sequence[value[2]] + prev_aligned_new_sequence
                    elif choice == 'l':
                        stack_aligned_reference_sequence =  '-' + prev_aligned_reference_sequence
                        stack_aligned_new_sequence =  new_sequence[value[2]] + prev_aligned_new_sequence
                    else:
                        raise Exception('error at choice')
                    stack.append((value[1], value[2], stack_aligned_reference_sequence, stack_aligned_new_sequence))
        

def load_blosum_matrix(filename):
    """ This function loads the blosum matrix from the given file. """
    blosum_matrix = []
    with open(filename,'r') as file:
        line_number=0
        for line in file:
            blosum_matrix.append([])
            entrys=line.strip('\n').split('\t')
            entry_number=0
            for entry in entrys:
                blosum_matrix[line_number].append(entry)
            line_number = line_number+1
    return blosum_matrix

In [106]:
allowed_characters = {'A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'}
reference_sequence = 'SKSYYDILGVPKSASERQIKKAFHKLAMKYHPDKNKSPDAEAKFREIAEAYETLSDANRRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSGQ'
new_sequence = 'KQDYYEILGVSKTAEEREIRKAYKRLAMKYHPDRNQGDKEAEAKFKEIKEAYEVLTDSQKRAAYDQYGHA'

print(f'len a={len(reference_sequence)}')
print(f'len b={len(new_sequence)}')

blosum_matrix = load_blosum_matrix('blosum.txt')

#print(blosum_matrix)

def similarity_score(symbol_a, symbol_b):
    j = blosum_matrix[0].index(symbol_a)
    i = blosum_matrix[0].index(symbol_b)
    return int(blosum_matrix[j][i])

align_sequences(reference_sequence=reference_sequence,
                new_sequence=new_sequence,
                allowed_characters=allowed_characters,
                gap_penalty=2,
                similarity_score_fn=similarity_score)
#reference_sequence = 'GCATGCU'
#new_sequence = 'GATTACA'  

len a=200
len b=70
> SK-SYYDILGVPKSAS-ERQI-KKAFH-KLAMKYHPDKNKSPDAE--AKFREIAEAYETLSDAN-RRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSG
> -KQDYYEILGVSKTA-EEREIR-KAY-KRLAMKYHPDRNQG-DKEAEAKFKEIKEAYEVLTDSQK-RAAYDQY---------------------------------G------------------------------------------------------------------------------H--------------------
similarity_score=191, best_indices=(1, 72)
simple_score=-59


> SK-SYYDILGVPKSAS-ERQI-KKAFHK-LAMKYHPDKNKSPDAE--AKFREIAEAYETLSDAN-RRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSG
> -KQDYYEILGVSKTA-EEREIR-KAY-KRLAMKYHPDRNQG-DKEAEAKFKEIKEAYEVLTDSQK-RAAYDQY---------------------------------G------------------------------------------------------------------------------H--------------------
similarity_score=194, best_indices=(1, 72)
simple_score=-56


> SK-SYYDILGVPKSAS-ER